#  NWB File generation from raw data: scaled up for one rat at a time
working July 2021
AC, adapted from RN

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Block usually prints "Resource 'XMLSchema.xsd' is already loaded"

import os
import logging
import sys
from rec_to_nwb.processing.builder.nwb_file_builder import NWBFileBuilder
from rec_to_binaries import extract_trodes_rec_file
from rec_to_nwb.processing.builder.raw_to_nwb_builder import RawToNWBBuilder
from rec_to_nwb.processing.metadata.metadata_manager import MetadataManager

## Setup config parameters
### see more details at rec_to_nwb repo readme or [here](https://novelaneuro.github.io/rec_to_nwb-docs/README.html#how-to-use-it)

In [ ]:
# Put your data path and animal name here
data_path = '/stelmo/your_name_dir/' #path holding data for all your rats 
animal_name = 'rat_name' #name of rat

# Outputs to nwb folder in your animal's directory for debugging
# Change this to 'stelmo/nwb/raw' for the actual conversion to shared franklab location
# Or, just keep nwbs in your own personal directory and put links in shared franklab nwb raw dir
output_path = os.path.join(data_path, animal_name, 'nwb/raw/')
# Change this to 'stelmo/nwb/video' for the actual conversion to shared franklab location
video_path = os.path.join(data_path, animal_name, 'nwb/video/')
# Path to metadata for each day of recording - either enter the shared repo location in your /home/your_name/Src/franklabnwb/yaml, or an alternative
yaml_path = os.path.join(data_path, animal_name, 'metadata/') 
# Probe metadata files should be in same dir as defined by your yaml_path
probe1_metadata_file = '128c-4s6mm6cm-15um-26um-sl.yml' #d2 in l&r mpfc ish for wilbur
probe2_metadata_file = 'tetrode_12.5.yml'
probe3_metadata_file = '128c-4s8mm6cm-20um-40um-sl.yml' #d1 in lofc ish for wilbur

# Get all dates from folders in data_path/animal_name/raw directory
raw_path = os.path.join(data_path, animal_name, 'raw/')
allDates = sorted([fld.name for fld in os.scandir(raw_path) if fld.is_dir()])

# If you have one master config for full animal, specify here, otherwise put a .trodesconf in each date dir 
# per the franklab file naming instructions to have a particular config used for each date of data
reconfig_file = animal_name + '_reconfig.xml'

In [ ]:
#inspect paths and dates
print(allDates)
print(output_path, raw_path, video_path, yaml_path, data_path)

## Run processing and create nwb file

In [ ]:
# currently does try loop for each date 
for date in allDates:
    print('trying to convert date: ', date)
    try:
        recording_session_path = os.path.join(data_path, animal_name, 'raw', date)
        # Only using one main config per rat, rather than individual configs, so reconfig is specified above
        reconfig_file = reconfig_file
        trodes_rec_export_args = ('-reconfig', os.path.join(yaml_path, reconfig_file)) 
        # Overwrite any existing preprocessing data?
        # Specify whether preprocessing data should be reextracted (True) or skipped if it already exists (False)
        overwrite = False
        # note that extraction will write to the directory: data_path+'preprocessing/' and
        # you may have data that you previously preprocessed in that dir, created with alternate export args
        # that you neither want to skip nor overwrite if you're using them for filterframework
        # so beware!!! One stratgy is to move filterframework based preprocessing files to a dir with another name
        animal_metadata_file = date + '_' + animal_name + '.yml'
        animal_metadata = os.path.join(yaml_path, animal_metadata_file )
        probe_metadata = [os.path.join(yaml_path, probe1_metadata_file), 
                          os.path.join(yaml_path, probe2_metadata_file),
                          os.path.join(yaml_path, probe3_metadata_file)
                         ]
        # Generate and print out metadata as a check
        metadata = MetadataManager(animal_metadata, probe_metadata)
        print(metadata)
        # Create NWB file!
        # Expect warnings from rec_to_binaries
        builder = RawToNWBBuilder(animal_name=animal_name,
                              data_path=data_path,
                              dates=[date],
                              nwb_metadata=metadata,
                              overwrite=overwrite,
                              output_path=output_path,
                              video_path=video_path,
                              trodes_rec_export_args = trodes_rec_export_args)
        content = builder.build_nwb()
        print(content) #just to look
        # Automatically delete preprocessing files? Comment out if you're in the debugging phase 
        builder.cleanup() #or
        del builder # clear the big variable 
    except Exception as e:
        print("error for date ", date," is: ", e)
        try:
            e.print_exception() #or print_tb() to get traceback? Or comment the except statement out, turn this all into "if" rather than "try" as you loop if easier to have automatic traceback output when error
        except:
            print("unable to print exception here, check the log file")
    else:
        print("no errors for date: ", date)
    finally:
        print("done trying to convert date: ", date)

In [ ]:
# open a file to inspect if you want
import pynwb
import ndx_franklab_novela
from nwbwidgets import nwb2widget
import matplotlib.pyplot as plt

In [ ]:
# set path for a raw data file
nwb_path = '/stelmo/your_name_dir/rat/nwb/raw/ratDATE.nwb'
io = pynwb.NWBHDF5IO(nwb_path, mode='r')

In [ ]:
# define nwb
nwb = io.read()

In [ ]:
# check it out
nwb2widget(nwb)

In [ ]:
# explore! For ex, with nwb.acquisition or nwb.electrodes, etc...
plt.figure(figsize=(30,10))
plt.plot(nwb.acquisition['e-series'].timestamps[1080000:1100000],nwb.acquisition['e-series'].data[1080000:1100000,5])
plt.show()

In [ ]:
# close when done
io.close()